Great. Here is your fully updated, clean, single-cell, production-ready code.

 **It includes:
RoBERTa-base
DeBERTa-v3-small**

ELECTRA-base-generator (the recommended ELECTRA variant for classification)

Professional text-only cleaning

Adversarial augmentation (20% of training data)

sentence shuffle

synonym replacement

random token masking

Independent training + saving for each model

In [ ]:
# -------------------------
# Full pipeline: cleaning, augmentation, paraphrasing, train 3 models, evaluate
# Paths and columns provided by user
# -------------------------

print("Installing dependencies (may take a minute)...")
!pip install -q transformers datasets accelerate torch scikit-learn nltk evaluate sentencepiece

# -------------------------
# Imports
# -------------------------
import os
import random
import time
import re
import math
import pandas as pd
import numpy as np
import torch
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoModelForSeq2SeqLM,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
import evaluate
import nltk
from nltk.corpus import wordnet

# -------------------------
# NLTK downloads
# -------------------------
nltk.download("wordnet")
nltk.download("omw-1.4")

# -------------------------
# Device
# -------------------------




Installing dependencies (may take a minute)...


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cuda


In [ ]:

# -------------------------
# User paths & columns
# -------------------------
FAKE_PATH = "/content/drive/MyDrive/Colab Notebooks/Fake.csv"
REAL_PATH = "/content/drive/MyDrive/Colab Notebooks/Real.csv"
TEXT_COL = "text"      # use text only
LABEL_COL = "label"    # label column

# Output model folders
BASE_OUT = "/content/drive/MyDrive/All_models"
os.makedirs(BASE_OUT, exist_ok=True)
MODEL_DIRS = {
    "roberta": os.path.join(BASE_OUT, "roberta_base"),
    "deberta": os.path.join(BASE_OUT, "deberta_v3_small"),
    "electra": os.path.join(BASE_OUT, "electra_base"),
}

# -------------------------
# Parameters (feel free to tweak)
# -------------------------
MAX_LEN = 192
BATCH_SIZE = 16
EPOCHS = 3
LEARNING_RATE = 2e-5
AUG_PERCENT = 0.20      # 20% adversarial augmentation
PARAPHRASE_PERCENT = 0.10  # 10% of training paraphrased via T5
PARAPHRASE_MODEL = "t5-base"
PARAPHRASE_BATCH = 8    # paraphrase batch size (smallish for colab)
RANDOM_SEED = 42

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

# -------------------------
# Utilities: cleaning + augmentation
# -------------------------
def strong_clean(text):
    """Apply the user's requested strong cleaning to text."""
    if pd.isna(text):
        return ""
    text = str(text).lower()
    # remove explicit words
    text = re.sub(r'\b(facebook|share|click|subscribe|viral|subscribe)\b', '', text)
    # remove non-word characters (keeping whitespace)
    text = re.sub(r'[^\w\s]', ' ', text)
    # remove numbers
    text = re.sub(r'\d+', ' ', text)
    # collapse whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def shuffle_sentences(text):
    parts = [s.strip() for s in re.split(r'(?<=[.!?])\s+', text) if s.strip()]
    if len(parts) <= 1:
        return text
    random.shuffle(parts)
    return " ".join(parts)

def synonym_replace(text):
    words = text.split()
    if len(words) < 5:
        return text
    idxs = list(range(len(words)))
    random_idx = random.choice(idxs)
    target = words[random_idx]
    synsets = wordnet.synsets(target)
    if not synsets:
        return text
    lemmas = synsets[0].lemma_names()
    if not lemmas:
        return text
    replacement = lemmas[0].replace('_', ' ')
    # Keep replacement only if it's not identical and is alphabetic
    if replacement.lower() != target.lower():
        words[random_idx] = replacement
    return " ".join(words)

def random_mask(text, mask_token="[MASK]", mask_prob=0.05):
    words = text.split()
    if len(words) < 5:
        return text
    for i in range(len(words)):
        if random.random() < mask_prob:
            words[i] = mask_token
    return " ".join(words)

def augment_text_simple(text):
    choice = random.choice(["shuffle", "synonym", "mask"])
    if choice == "shuffle":
        return shuffle_sentences(text)
    elif choice == "synonym":
        return synonym_replace(text)
    else:
        return random_mask(text)


In [ ]:

# -------------------------
# 1. Load raw CSVs, prepare DataFrame (text only)
# -------------------------
print("Loading datasets...")
fake = pd.read_csv(FAKE_PATH)
real = pd.read_csv(REAL_PATH)

# Ensure label present: user said files contain label column; but if not, assign.
if LABEL_COL not in fake.columns:
    fake[LABEL_COL] = 0
if LABEL_COL not in real.columns:
    real[LABEL_COL] = 1

fake[LABEL_COL] = 0
real[LABEL_COL] = 1

# Use text only (ignore title)
fake_texts = fake[[TEXT_COL, LABEL_COL]].rename(columns={TEXT_COL: "text"})
real_texts = real[[TEXT_COL, LABEL_COL]].rename(columns={TEXT_COL: "text"})

df = pd.concat([fake_texts, real_texts], ignore_index=True)
print("Raw combined shape:", df.shape)

# Clean text strongly
print("Applying strong cleaning...")
df["text"] = df["text"].fillna("").apply(str).apply(str.strip).apply(str)
df["text"] = df["text"].apply(str).apply(lambda t: strong_clean(t))

# Remove too short/empty examples
min_len = 50
before = len(df)
df = df[df["text"].str.len() > min_len].reset_index(drop=True)
after = len(df)
print(f"Removed {before-after} short/empty rows; final size {after}")

# Shuffle
df = df.sample(frac=1, random_state=RANDOM_SEED).reset_index(drop=True)

# -------------------------
# 2. Train/Val/Test split (stratified)
# -------------------------
print("Splitting data...")
train_val_df, test_df = train_test_split(df, test_size=0.2, stratify=df[LABEL_COL], random_state=RANDOM_SEED)
train_df, val_df = train_test_split(train_val_df, test_size=(0.1/0.8), stratify=train_val_df[LABEL_COL], random_state=RANDOM_SEED)

print("Train:", len(train_df), "Val:", len(val_df), "Test:", len(test_df))

# -------------------------
# 3. Adversarial augmentation (20% of train)
# -------------------------
print(f"Applying adversarial augmentation ({int(AUG_PERCENT*100)}%) to training set...")
aug_n = int(len(train_df) * AUG_PERCENT)
if aug_n > 0:
    aug_sample = train_df.sample(n=aug_n, random_state=RANDOM_SEED).copy()
    aug_sample["text"] = aug_sample["text"].apply(augment_text_simple)
    train_df = pd.concat([train_df, aug_sample], ignore_index=True).sample(frac=1, random_state=RANDOM_SEED).reset_index(drop=True)
print("Train size after augmentation:", len(train_df))

# -------------------------
# 4. Paraphrase ~10% of train using T5 (in-place replacement of those rows)
# -------------------------
PARAPHRASE_N = int(len(train_df) * PARAPHRASE_PERCENT)
if PARAPHRASE_N > 0:
    print(f"Paraphrasing {PARAPHRASE_N} training rows with {PARAPHRASE_MODEL} (this may take a few minutes)...")
    par_tok = AutoTokenizer.from_pretrained(PARAPHRASE_MODEL)
    par_model = AutoModelForSeq2SeqLM.from_pretrained(PARAPHRASE_MODEL).to(device)
    indices = train_df.sample(n=PARAPHRASE_N, random_state=RANDOM_SEED).index.tolist()
    # process in batches
    for start in range(0, len(indices), PARAPHRASE_BATCH):
        batch_idx = indices[start:start+PARAPHRASE_BATCH]
        texts = train_df.loc[batch_idx, "text"].tolist()
        inputs = ["paraphrase: " + t for t in texts]
        tok = par_tok(inputs, return_tensors="pt", padding=True, truncation=True, max_length=MAX_LEN).to(device)
        with torch.no_grad():
            gen = par_model.generate(**tok,
                                     max_length=MAX_LEN,
                                     num_beams=5,
                                     do_sample=True,
                                     top_k=50,
                                     top_p=0.95,
                                     temperature=0.7,
                                     early_stopping=True)
        outs = par_tok.batch_decode(gen, skip_special_tokens=True, clean_up_tokenization_spaces=True)
        # replace (if lengths match)
        for i, idx in enumerate(batch_idx):
            if i < len(outs):
                new_text = strong_clean(outs[i])
                # keep paraphrase only if it is reasonably different and not empty
                if len(new_text) > 20:
                    train_df.at[idx, "text"] = new_text
    # free paraphrase model
    del par_model
    torch.cuda.empty_cache()
    print("Paraphrasing done.")

# -------------------------
# 5. Convert to HuggingFace Datasets
# -------------------------
train_ds = Dataset.from_pandas(train_df.reset_index(drop=True))
val_ds = Dataset.from_pandas(val_df.reset_index(drop=True))
test_ds = Dataset.from_pandas(test_df.reset_index(drop=True))


Loading datasets...


/tmp/ipython-input-4004363255.py:5: DtypeWarning: Columns (4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171) have mixed types. Specify dtype option on import or set low_memory=False.
  fake = pd.read_csv(FAKE_PATH)


Raw combined shape: (44940, 2)
Applying strong cleaning...
Removed 876 short/empty rows; final size 44064
Splitting data...
Train: 30844 Val: 4407 Test: 8813
Applying adversarial augmentation (20%) to training set...
Train size after augmentation: 37012
Paraphrasing 3701 training rows with t5-base (this may take a few minutes)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Paraphrasing done.


In [ ]:

# -------------------------
# 6. Training routine for each model
# -------------------------
acc_metric = evaluate.load("accuracy")

def train_and_save(model_id, out_dir):
    print("\n" + "="*60)
    print("TRAINING MODEL:", model_id)
    print("Saving to:", out_dir)
    print("="*60)
    os.makedirs(out_dir, exist_ok=True)

    tokenizer = AutoTokenizer.from_pretrained(model_id, do_lower_case=True)

    def tokenize_batch(batch):
        return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=MAX_LEN)

    # map tokenization
    ds_train = train_ds.map(tokenize_batch, batched=True)
    ds_val = val_ds.map(tokenize_batch, batched=True)
    ds_test = test_ds.map(tokenize_batch, batched=True)

    # remove original text columns and rename label -> labels
    for d in [ds_train, ds_val, ds_test]:
        cols = [c for c in d.column_names if c not in ["input_ids", "attention_mask", "labels"]]
        # ensure label present
    ds_train = ds_train.remove_columns([c for c in ds_train.column_names if c not in ["input_ids", "attention_mask", "label"]])
    ds_val = ds_val.remove_columns([c for c in ds_val.column_names if c not in ["input_ids", "attention_mask", "label"]])
    ds_test = ds_test.remove_columns([c for c in ds_test.column_names if c not in ["input_ids", "attention_mask", "label"]])

    ds_train = ds_train.rename_column("label", "labels")
    ds_val = ds_val.rename_column("label", "labels")
    ds_test = ds_test.rename_column("label", "labels")

    ds_train.set_format("torch")
    ds_val.set_format("torch")
    ds_test.set_format("torch")

    model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=2).to(device)

    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        preds = np.argmax(logits, axis=1)
        return acc_metric.compute(predictions=preds, references=labels)

    args = TrainingArguments(
        output_dir=out_dir,
        eval_strategy="epoch",
        save_strategy="epoch",
        learning_rate=LEARNING_RATE,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        num_train_epochs=EPOCHS,
        weight_decay=0.01,
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        logging_steps=200,
        report_to="none",
        save_total_limit=2,
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=ds_train,
        eval_dataset=ds_val,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=1)]
    )

    # train
    start = time.time()
    trainer.train()
    elapsed = time.time() - start
    print(f"Training finished in {elapsed/60:.1f} minutes")

    # evaluate on test
    print("Evaluating on test set...")
    pred_out = trainer.predict(ds_test)
    y_true = pred_out.label_ids
    y_pred = np.argmax(pred_out.predictions, axis=1)
    print("Accuracy:", accuracy_score(y_true, y_pred))
    print(classification_report(y_true, y_pred, target_names=["Fake", "Real"]))

    # save model + tokenizer
    trainer.save_model(out_dir)
    tokenizer.save_pretrained(out_dir)

    # cleanup to free GPU
    del model
    del trainer
    torch.cuda.empty_cache()

    return {"y_true": y_true, "y_pred": y_pred, "out_dir": out_dir}


In [ ]:

# -------------------------
# 7. Train all three models
# -------------------------
results = {}
models_to_train = [
    ("roberta-base", MODEL_DIRS["roberta"]),
    ("microsoft/deberta-v3-small", MODEL_DIRS["deberta"]),
    ("google/electra-base-generator", MODEL_DIRS["electra"])
]

for model_id, out_dir in models_to_train:
    info = train_and_save(model_id, out_dir)
    results[model_id] = info



TRAINING MODEL: roberta-base
Saving to: /content/drive/MyDrive/All_models/roberta_base


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-generator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-1186065081.py:63: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.048100,0.013747,0.997277
2,0.039200,0.008084,0.998639
3,0.017600,0.008884,0.998639


model.safetensors:   0%|          | 0.00/135M [00:00<?, ?B/s]

Training finished in 12.4 minutes
Evaluating on test set...


Accuracy: 0.9984114376489277
              precision    recall  f1-score   support

        Fake       1.00      1.00      1.00      4530
        Real       1.00      1.00      1.00      4283

    accuracy                           1.00      8813
   macro avg       1.00      1.00      1.00      8813
weighted avg       1.00      1.00      1.00      8813



In [ ]:

# -------------------------
# 8. Paraphrase-robustness evaluation (paraphrase 200 test samples and evaluate)
# -------------------------
print("\nParaphrase robustness test: paraphrasing 200 test samples using T5 and evaluating models...")

PARAPH_TEST_N = 200
par_model_name = PARAPHRASE_MODEL
par_tok = AutoTokenizer.from_pretrained(par_model_name)
par_model = AutoModelForSeq2SeqLM.from_pretrained(par_model_name).to(device)

sampled = test_df.sample(n=min(PARAPH_TEST_N, len(test_df)), random_state=RANDOM_SEED).reset_index(drop=True)
orig_texts = sampled["text"].tolist()
orig_labels = sampled[LABEL_COL].tolist()

def paraphrase_texts(texts, batch_size=PARAPHRASE_BATCH):
    paraphrases = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        inputs = ["paraphrase: " + t for t in batch]
        tok = par_tok(inputs, return_tensors="pt", padding=True, truncation=True, max_length=MAX_LEN).to(device)
        with torch.no_grad():
            gen = par_model.generate(**tok,
                                     max_length=MAX_LEN,
                                     num_beams=5,
                                     do_sample=True,
                                     top_k=50,
                                     top_p=0.95,
                                     temperature=0.7,
                                     early_stopping=True)
        outs = par_tok.batch_decode(gen, skip_special_tokens=True, clean_up_tokenization_spaces=True)
        paraphrases.extend(outs[:len(batch)])
    return paraphrases

par_texts = paraphrase_texts(orig_texts)
par_texts = [strong_clean(t) for t in par_texts]

# helper to predict with saved model directories
def predict_with_model_dir(model_dir, texts, bs=32):
    tok = AutoTokenizer.from_pretrained(model_dir)
    model = AutoModelForSequenceClassification.from_pretrained(model_dir).to(device)
    preds = []
    for i in range(0, len(texts), bs):
        batch = texts[i:i+bs]
        enc = tok(batch, return_tensors="pt", padding=True, truncation=True, max_length=MAX_LEN).to(device)
        with torch.no_grad():
            out = model(**enc)
            logits = out.logits.cpu().numpy()
            batch_preds = np.argmax(logits, axis=1)
            preds.extend(batch_preds.tolist())
    del model
    torch.cuda.empty_cache()
    return preds



Paraphrase robustness test: paraphrasing 200 test samples using T5 and evaluating models...


In [ ]:

# evaluate each saved model on paraphrased samples
summary_rows = []
for short_name, model_dir in MODEL_DIRS.items():
    if not os.path.isdir(model_dir):
        print("Missing model folder:", model_dir)
        continue
    print("\nEvaluating saved model:", short_name)
    y_pred = predict_with_model_dir(model_dir, par_texts, bs=32)
    acc = accuracy_score(orig_labels, y_pred)
    print(f"Accuracy on paraphrased {len(par_texts)} samples: {acc*100:.2f}%")
    print(classification_report(orig_labels, y_pred, target_names=["Fake", "Real"]))
    summary_rows.append({"model": short_name, "paraphrase_accuracy": acc})

# Save summary CSV
summary_df = pd.DataFrame(summary_rows)
summary_path = "/content/paraphrase_robustness_summary.csv"
summary_df.to_csv(summary_path, index=False)
print("\nSaved paraphrase robustness summary to:", summary_path)

# -------------------------
# Final message
# -------------------------
print("\nALL DONE.")
print("Models saved at:")
for k,v in MODEL_DIRS.items():
    print(k, "->", v)
print("Paraphrase robustness summary:", summary_path)


Evaluating saved model: roberta
Accuracy on paraphrased 200 samples: 89.00%
              precision    recall  f1-score   support

        Fake       0.94      0.85      0.89       104
        Real       0.85      0.94      0.89        96

    accuracy                           0.89       200
   macro avg       0.89      0.89      0.89       200
weighted avg       0.89      0.89      0.89       200


Evaluating saved model: deberta
Accuracy on paraphrased 200 samples: 89.50%
              precision    recall  f1-score   support

        Fake       0.89      0.91      0.90       104
        Real       0.90      0.88      0.89        96

    accuracy                           0.90       200
   macro avg       0.90      0.89      0.89       200
weighted avg       0.90      0.90      0.89       200


Evaluating saved model: electra
Accuracy on paraphrased 200 samples: 91.50%
              precision    recall  f1-score   support

        Fake       0.91      0.92      0.92       104
      